# 🦜🔗 RAG Fundamentals with LangChain & Groq

Welcome to **RAG (Retrieval Augmented Generation) Fundamentals**! This notebook teaches you how to build a RAG pipeline using LangChain and the **free Groq API**.

## What is RAG?
RAG combines **retrieval** (finding relevant documents) with **generation** (LLM response). Instead of relying only on the LLM's training data, we provide context from our own documents.

```
User Query → Retrieve Relevant Docs → Augment Prompt with Context → Generate Answer
```

## What You'll Learn:
1. **Document Loaders** - Load PDFs and other documents
2. **Text Splitters** - Chunk documents for processing
3. **Embeddings** - Convert text to vectors
4. **Vector Stores** - Store and search embeddings
5. **Retrievers** - Find relevant documents
6. **RAG Chains** - Build end-to-end question answering

## Dataset
We'll use **Landmark Judgments of the Supreme Court of India** PDF to build a legal Q&A system.

---

## Prerequisites
- Complete the LangChain Fundamentals notebook first
- Groq API key in your `.env` file

## 1. Setup & Installation

Install the required packages for RAG:

In [ ]:
# Install required packages (uncomment if needed)
# %pip install langchain langchain-groq langchain-huggingface langchain-community -q
# %pip install pypdf faiss-cpu sentence-transformers -q

In [6]:
# Import required libraries
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify API key
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    print("⚠️ GROQ_API_KEY not found! Set it in your .env file or uncomment below:")
    # os.environ["GROQ_API_KEY"] = "your-groq-api-key-here"
else:
    print("✅ GROQ_API_KEY loaded successfully!")

✅ GROQ_API_KEY loaded successfully!


In [7]:
# Initialize Groq LLM
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,  # Lower temperature for factual responses
    max_tokens=1024,
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: llama-3.3-70b-versatile


## 2. Document Loaders

**Document Loaders** load data from various sources (PDFs, web pages, databases) into a standard `Document` format.

Each `Document` has:
- `page_content`: The text content
- `metadata`: Information about the source (page number, file name, etc.)

In [8]:
from langchain_community.document_loaders import PyPDFLoader

# Load the Supreme Court Landmark Judgments PDF
pdf_path = "data/AMZN-Q3-2025-Earnings-Release.pdf"
loader = PyPDFLoader(pdf_path)

# Load all pages as documents
documents = loader.load()

print(f"📄 Loaded {len(documents)} pages from the PDF")
print(f"\n--- First Page Preview ---")
print(f"Content (first 500 chars): {documents[0].page_content[:500]}...")
print(f"\nMetadata: {documents[0].metadata}")

📄 Loaded 13 pages from the PDF

--- First Page Preview ---
Content (first 500 chars): AMAZON.COM ANNOUNCES THIRD QUARTER RESULTS
SEATTLE—(BUSINESS WIRE) October 30, 2025—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial 
results for its third quarter ended September 30, 2025. 
• Net sales increased 13% to $180.2 billion in the third quarter, compared with $158.9 billion in third quarter 2024.
Excluding the $1.5 billion favorable impact from year-over-year changes in foreign exchange rates throughout the
quarter, net sales increased 12% compared with third quarter 2024.
• ...

Metadata: {'producer': 'Wdesk Fidelity Content Translations Version 014.006.087', 'creator': 'Workiva', 'creationdate': '2025-10-30T16:57:42+00:00', 'author': 'anonymous', 'moddate': '2025-10-30T09:59:42-07:00', 'title': 'AMZN-2025.09.30-EX99.1', 'source': 'data/AMZN-Q3-2025-Earnings-Release.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}


## 3. Text Splitters

LLMs have **context length limits**. We need to split documents into smaller **chunks** that:
- Fit within the model's context window
- Contain semantically meaningful content
- Have some overlap to maintain context between chunks

### Key Parameters:
- `chunk_size`: Maximum characters per chunk
- `chunk_overlap`: Characters shared between consecutive chunks

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Max characters per chunk
    chunk_overlap=200,    # Overlap between chunks for context
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]  # Split priorities
)

# Split documents into chunks
chunks = text_splitter.split_documents(documents)

print(f"📦 Split {len(documents)} pages into {len(chunks)} chunks")
print(f"\n--- Sample Chunk ---")
print(f"Chunk 5 content: {chunks[5].page_content[:300]}...")
print(f"Chunk 5 metadata: {chunks[5].metadata}")

📦 Split 13 pages into 46 chunks

--- Sample Chunk ---
Chunk 5 content: Some other highlights since the company’s last earnings announcement include that Amazon:
• Saw continued strong adoption of Trainium2, its custom AI chip, which is fully subscribed and a multi-billion-dollar 
business that grew 150% quarter over quarter. 
• Launched Project Rainier, a massive AI co...
Chunk 5 metadata: {'producer': 'Wdesk Fidelity Content Translations Version 014.006.087', 'creator': 'Workiva', 'creationdate': '2025-10-30T16:57:42+00:00', 'author': 'anonymous', 'moddate': '2025-10-30T09:59:42-07:00', 'title': 'AMZN-2025.09.30-EX99.1', 'source': 'data/AMZN-Q3-2025-Earnings-Release.pdf', 'total_pages': 13, 'page': 1, 'page_label': '2'}


## 4. Embeddings

**Embeddings** convert text into numerical vectors that capture semantic meaning. Similar texts have similar vectors.

```
"The court ruled in favor" → [0.23, -0.45, 0.67, ...]
"The judgment was positive" → [0.21, -0.43, 0.65, ...]  ← Similar vector!
```

We'll use **HuggingFace embeddings** (free, runs locally).

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings

# Initialize embeddings model (free, runs locally)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  # Fast and efficient
    model_kwargs={'device': 'cpu'},  # Use 'cuda' if you have GPU
)

# Test the embeddings
sample_text = "The Supreme Court of India"
embedding_vector = embeddings.embed_query(sample_text)

print(f"✅ Embeddings model loaded: all-MiniLM-L6-v2")
print(f"📐 Embedding dimension: {len(embedding_vector)}")
print(f"🔢 First 5 values: {embedding_vector[:5]}")

✅ Embeddings model loaded: all-MiniLM-L6-v2
📐 Embedding dimension: 384
🔢 First 5 values: [-0.05370708182454109, 0.05493843927979469, -0.07679767161607742, -0.050161395221948624, -0.06389239430427551]


## 5. Vector Stores

**Vector Stores** are databases optimized for storing and searching embeddings. They enable fast **similarity search**.

Popular options:
- **FAISS** (Facebook AI) - Fast, in-memory, free
- **Chroma** - Easy to use, persistent storage
- **Pinecone** - Cloud-based, scalable

We'll use **FAISS** (free and fast).

In [11]:
from langchain_community.vectorstores import FAISS

# Create vector store from document chunks
# This embeds all chunks and stores them in FAISS
vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)

print(f"✅ Vector store created with {len(chunks)} vectors")
print(f"📊 Index type: FAISS (in-memory)")

✅ Vector store created with 46 vectors
📊 Index type: FAISS (in-memory)


In [18]:
# Test similarity search
query = "What is the Amazon AI Revenue"
similar_docs = vectorstore.similarity_search(query, k=3)

print(f"🔍 Query: '{query}'")
print(f"\n📄 Top 3 similar documents:")
for i, doc in enumerate(similar_docs, 1):
    print(f"\n--- Document {i} (Page {doc.metadata.get('page', 'N/A')}) ---")
    print(doc.page_content[:300] + "...")

🔍 Query: 'What is the Amazon AI Revenue'

📄 Top 3 similar documents:

--- Document 1 (Page 0) ---
• Operating cash flow increased 16% to $130.7 billion for the trailing twelve months, compared with $112.7 billion for
the trailing twelve months ended September 30, 2024.
• Free cash flow decreased to $14.8 billion for the trailing twelve months, driven primarily by a year-over-year increase
of $50...

--- Document 2 (Page 7) ---
AMAZON.COM, INC.
Segment Information
(in millions)
(unaudited) 
  
Three Months Ended
September 30,
Nine Months Ended
September 30,
2024 2025 2024 2025
North America
Net sales $ 95,537 $ 106,267 $ 271,911 $ 299,222 
Operating expenses  89,874  101,478  256,200  281,075 
Operating income $ 5,663 $ 4,...

--- Document 3 (Page 5) ---
AMAZON.COM, INC.
Consolidated Statements of Operations
(in millions, except per share data)
(unaudited)
  
Three Months Ended
September 30,
Nine Months Ended
September 30,
2024 2025 2024 2025
Net product sales $ 67,601 $ 74,058 $ 190,08

## 6. Retrievers

A **Retriever** is an interface that returns relevant documents given a query. Vector stores can be converted to retrievers.

### Retriever Types:
- **Similarity** - Returns top k most similar documents
- **MMR (Maximum Marginal Relevance)** - Balances relevance with diversity
- **Self-Query** - LLM generates filter queries

In [13]:
# Convert vector store to retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",  # or "mmr" for diversity
    search_kwargs={"k": 4}     # Return top 4 documents
)

# Test retriever
query = "What are AWS AI Investement plan?"
retrieved_docs = retriever.invoke(query)

print(f"🔍 Query: '{query}'")
print(f"📄 Retrieved {len(retrieved_docs)} documents")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n--- Doc {i} ---")
    print(doc.page_content[:200] + "...")

🔍 Query: 'What are AWS AI Investement plan?'
📄 Retrieved 4 documents

--- Doc 1 ---
• Operating cash flow increased 16% to $130.7 billion for the trailing twelve months, compared with $112.7 billion for
the trailing twelve months ended September 30, 2024.
• Free cash flow decreased t...

--- Doc 2 ---
employees interact with AI agents that can find insights, conduct research, and take actions across systems. Quick 
Suite helps turn month-long projects into days, get 80%+ time savings on complex tas...

--- Doc 3 ---
Qwen3, as well as Anthropic’s Claude Sonnet 4.5, Claude Opus 4.1, and Claude Haiku 4.5.
• Announced a number of new Amazon EC2 instances for general purpose computing, including AWS Graviton4 chips 
f...

--- Doc 4 ---
Some other highlights since the company’s last earnings announcement include that Amazon:
• Saw continued strong adoption of Trainium2, its custom AI chip, which is fully subscribed and a multi-billio...


## 7. RAG Chain - Putting It All Together

Now we combine everything into a **RAG Chain**:

```
Query → Retriever → Relevant Docs → Prompt + Context → LLM → Answer
```

We'll create a prompt that instructs the LLM to answer based ONLY on the provided context.

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create RAG prompt template
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful financial assistant specializing in Stock Market investment.
Answer the question based ONLY on the following context from  Amazon Q3 2025 Eearning Results.
If the answer is not found in the context, say "I cannot find this information in the provided documents."

Context:
{context}"""),
    ("human", "{question}")
])

print("✅ RAG prompt template created")

✅ RAG prompt template created


In [19]:
# Helper function to format retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Build the RAG chain using LCEL
rag_chain = (
    {
        "context": retriever | format_docs,  # Retrieve and format docs
        "question": RunnablePassthrough()     # Pass through the question
    }
    | rag_prompt     # Create the prompt
    | llm            # Generate response
    | StrOutputParser()  # Parse to string
)

print("✅ RAG chain created!")

✅ RAG chain created!


## 8. Test the RAG System

Let's ask questions about Indian Supreme Court landmark judgments!

In [22]:
# Question 1: Kesavananda Bharati Case
question = "What is the AWS Revenue?"

print(f"❓ Question: {question}")
print("\n" + "="*50)
answer = rag_chain.invoke(question)
print(f"\n💡 Answer:\n{answer}")

❓ Question: What is the AWS Revenue?


💡 Answer:
The AWS revenue for the three months ended September 30, 2025, is $33,006 million, and for the nine months ended September 30, 2025, it is $93,146 million.


## 9. RAG with Source Attribution

Good RAG systems show **which documents** were used to generate the answer. Let's create a version that returns sources.

In [24]:
from langchain_core.runnables import RunnableParallel

# Create a chain that returns both answer and source documents
rag_chain_with_sources = RunnableParallel(
    {
        "answer": rag_chain,
        "sources": retriever
    }
)

# Test with sources
question = "What is AWS Revenue?"
result = rag_chain_with_sources.invoke(question)

print(f"❓ Question: {question}")
print("\n" + "="*50)
print(f"\n💡 Answer:\n{result['answer']}")
print("\n" + "="*50)
print(f"\n📚 Sources used:")
for i, doc in enumerate(result['sources'], 1):
    page = doc.metadata.get('page', 'N/A')
    print(f"  {i}. Page {page}: {doc.page_content[:100]}...")

❓ Question: What is AWS Revenue?


💡 Answer:
According to the provided context, AWS revenue for the:

- Three Months Ended September 30, 2024: $27,452 million
- Three Months Ended September 30, 2025: $33,006 million
- Nine Months Ended September 30, 2024: $78,770 million
- Nine Months Ended September 30, 2025: $93,146 million


📚 Sources used:
  1. Page 7: AMAZON.COM, INC.
Segment Information
(in millions)
(unaudited) 
  
Three Months Ended
September 30,
...
  2. Page 0: • Operating cash flow increased 16% to $130.7 billion for the trailing twelve months, compared with ...
  3. Page 5: AMAZON.COM, INC.
Consolidated Statements of Operations
(in millions, except per share data)
(unaudit...
  4. Page 7: Operating income  17,411  17,422  47,390  54,998 
Total non-operating income (expense)  626  10,748 ...


## 10. Streaming RAG Responses

For better user experience, stream the response token by token:

In [26]:
# Stream the response
question = "What is AWS  Trainium2?"

print(f"❓ Question: {question}")
print("\n💡 Streaming Answer:")
print("-" * 40)

for chunk in rag_chain.stream(question):
    print(chunk, end="", flush=True)

print("\n" + "-" * 40)

❓ Question: What is AWS  Trainium2?

💡 Streaming Answer:
----------------------------------------
According to the context, Trainium2 is Amazon's custom AI chip. It is a multi-billion-dollar business that grew 150% quarter over quarter and is fully subscribed. Additionally, nearly 500,000 Trainium2 chips are being used in Project Rainier, a massive AI compute cluster, to build and deploy Anthropic's leading Claude AI models.
----------------------------------------


## 11. Save & Load Vector Store

Save the vector store to disk so you don't have to re-embed documents each time:

In [27]:
# Save vector store to disk
vectorstore.save_local("faiss_supreme_court_index")
print("✅ Vector store saved to 'faiss_supreme_court_index'")

# Load it back (useful for future sessions)
# loaded_vectorstore = FAISS.load_local(
#     "faiss_supreme_court_index",
#     embeddings,
#     allow_dangerous_deserialization=True
# )
# print("✅ Vector store loaded from disk")

✅ Vector store saved to 'faiss_supreme_court_index'


## 12. Interactive Q&A Function

Create a reusable function for asking questions:

In [28]:
def ask_financial_question(question: str, show_sources: bool = True):
    """
    Ask a question about Indian Supreme Court landmark judgments.

    Args:
        question: Your legal question
        show_sources: Whether to show source documents
    """
    print(f"❓ Question: {question}")
    print("=" * 60)

    if show_sources:
        result = rag_chain_with_sources.invoke(question)
        print(f"\n💡 Answer:\n{result['answer']}")
        print("\n" + "=" * 60)
        print("\n📚 Sources:")
        for i, doc in enumerate(result['sources'], 1):
            page = doc.metadata.get('page', 'N/A')
            print(f"  [{i}] Page {page}")
    else:
        answer = rag_chain.invoke(question)
        print(f"\n💡 Answer:\n{answer}")

    print("\n" + "=" * 60)

# Example usage
ask_financial_question("What is AWS Revenue?")

❓ Question: What is AWS Revenue?

💡 Answer:
According to the provided context, AWS revenue for the:

- Three Months Ended September 30, 2024: $27,452 million
- Three Months Ended September 30, 2025: $33,006 million
- Nine Months Ended September 30, 2024: $78,770 million
- Nine Months Ended September 30, 2025: $93,146 million


📚 Sources:
  [1] Page 7
  [2] Page 0
  [3] Page 5
  [4] Page 7



## 🎯 Summary

In this notebook, you learned the **RAG Fundamentals**:

| Component | What You Learned |
|-----------|------------------|
| **Document Loaders** | Load PDFs with `PyPDFLoader` |
| **Text Splitters** | Chunk documents with `RecursiveCharacterTextSplitter` |
| **Embeddings** | Convert text to vectors with `HuggingFaceEmbeddings` |
| **Vector Stores** | Store and search with `FAISS` |
| **Retrievers** | Find relevant documents with similarity search |
| **RAG Chains** | Combine retrieval + generation with LCEL |
| **Source Attribution** | Return sources alongside answers |
| **Streaming** | Stream responses for better UX |
| **Persistence** | Save/load vector stores |

---

### RAG Pipeline Architecture:

```
┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│  PDF/Docs   │───▶│   Chunks    │───▶│  Embeddings │
└─────────────┘    └─────────────┘    └─────────────┘
                                            │
                                            ▼
┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│   Answer    │◀───│     LLM     │◀───│ Vector Store│
└─────────────┘    └─────────────┘    └─────────────┘
                          ▲                  │
                          │                  ▼
                   ┌─────────────┐    ┌─────────────┐
                   │   Prompt    │◀───│  Retriever  │◀── Query
                   └─────────────┘    └─────────────┘
```

### Next Steps:
- Add more document types (web pages, APIs)
- Experiment with different chunk sizes
- Try different embedding models
- Add conversation memory for multi-turn chat